In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, parser='auto')
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist["data"], mnist["target"]

In [4]:
y = y.astype(np.uint8)

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [6]:
print(y.shape)
print(np.array(y).reshape(-1,1).shape)
y.unique()
# 아래 구문에서 오류 발생: Y = enc.transform(y[:,np.newaxis]).toarray()
# 오류: ValueError: Multi-dimensional indexing (e.g. `obj[:, None]`) is no longer supported. Convert to a numpy array before indexing instead.
# 오류 해결을 위해 np.array로 변환 후  수정 (이후 1곳도 똑같이 수정)

(70000,)
(70000, 1)


array([5, 0, 4, 1, 9, 2, 3, 6, 7, 8], dtype=uint8)

In [7]:
enc.fit(np.array(y).reshape(-1,1))

OneHotEncoder()

In [8]:
# one-hot encoder 정보
print("category count:",enc.n_features_in_)
print(enc.categories_)

category count: 1
[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)]


In [9]:
Y = enc.transform(np.array(y).reshape(-1,1)).toarray()

In [10]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], Y[:60000], Y[60000:]

In [11]:
X_train = X_train / 255
X_test = X_test / 255

In [12]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [13]:
def softmax(X, W):
    K = np.size(W, 1)
    A = np.exp(X @ W)
    B = np.diag(1 / (np.reshape(A @ np.ones((K,1)), -1)))
    Y = B @ A
    return Y

In [14]:
def compute_cost(X, T, W):
    epsilon = 1e-5
    N = len(T)
    K = np.size(T, 1)
    cost = - (1/N) * np.ones((1,N)) @ (np.multiply(np.log(softmax(X, W) + epsilon), T)) @ np.ones((K,1))
    return cost

In [15]:
def predict(X, W):
    return np.argmax((X @ W), axis=1)

In [16]:
def batch_gd(X, T, W, learning_rate, iterations, batch_size):
    N = len(T)
    cost_history = np.zeros((iterations,1))
    shuffled_indices = np.random.permutation(N)
    X_shuffled = X[shuffled_indices]
    T_shuffled = T[shuffled_indices]

    for i in range(iterations):
        j = i % N
        X_batch = X_shuffled[j:j+batch_size]
        T_batch = T_shuffled[j:j+batch_size]
        # batch가 epoch 경계를 넘어가는 경우, 앞 부분으로 채워줌
        if X_batch.shape[0] < batch_size:
            X_batch = np.vstack((X_batch, X_shuffled[:(batch_size - X_batch.shape[0])]))
            T_batch = np.vstack((T_batch, T_shuffled[:(batch_size - T_batch.shape[0])]))
        W = W - (learning_rate/batch_size) * (X_batch.T @ (softmax(X_batch, W) - T_batch))
        cost_history[i] = compute_cost(X_batch, T_batch, W)
        if i % 1000 == 0:
            print(cost_history[i][0])

    return (cost_history, W)

In [17]:
X = np.hstack((np.ones((np.size(X_train, 0),1)),X_train))
T = y_train

K = np.size(T, 1)
M = np.size(X, 1)
W = np.zeros((M,K))

iterations = 50000
learning_rate = 0.01

initial_cost = compute_cost(X, T, W)

print("Initial Cost is: {} \n".format(initial_cost[0][0]))

(cost_history, W_optimal) = batch_gd(X, T, W, learning_rate, iterations, 64)

Initial Cost is: 2.3024850979937166 

2.275899551989903
0.519605188443927
0.33949684821757165
0.4275669540908364
0.28558994587667574
0.41541993928413495
0.29507326049745636
0.29611068042166067
0.3238650907249443
0.44196697723264106
0.3136951609936433
0.25446000399323865
0.3284127031543774
0.3575932751396952
0.17404459107763504
0.40981621749268793
0.3485632846419652
0.3234320513050596
0.3459217547446731
0.21196261991814241
0.3555266150196701
0.415371811848336
0.24793104122551984
0.32566147372724646
0.17171222523504404
0.22905907261539896
0.28347789906066123
0.31041069967727764
0.16812312841401525
0.3950326473007408
0.3799075909097027
0.20925251798298536
0.17758781115989322
0.3702719355200841
0.33283828712113983
0.1140764808342393
0.276524055717379
0.14851625533459914
0.17704282624707596
0.37413636935395905
0.1389755231790908
0.25394405945600473
0.10764712404644428
0.21429152973836224
0.371646484697546
0.3572729063415555
0.2103456678678311
0.33946292711894316
0.30655763168325745
0.161600

In [18]:
## Accuracy
X_ = np.hstack((np.ones((np.size(X_test, 0),1)),X_test))
T_ = y_test
y_pred = predict(X_, W_optimal)
score = float(sum(y_pred == np.argmax(T_, axis=1)))/ float(len(y_test))

print(score)

0.9173
